In [69]:
import numpy as np 
import random
#定义一个基础的神经网络类
class neuralNetWork(object):
    
    def __init__(self, sizes):
        """
        Args: 
            sizes: list 相应的数值代指各层神经元的数量;
            weights: 神经元之间的权重
            biases: 神经元的偏置
            weights和biases参数都以标准正态分布初始化;
        """
        self.sizes = sizes
        self.num_layers = len(sizes)
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
    
    def feedforward(self, a):
        """
        返回神经元的输出，a是输入层的输入数据
        """
        for w, b in zip(self.weights, self.biases):
            a = sigmoid(np.dot(w, a) + b)
        return a
    
    def SGD(self, train, epochs, min_batch_size, eta, test=None):
        """
        用小批量随机梯度下降算法训练神经网络。
        train: list, 每个元素是(x, y)形式的tuple， x代表输入数据特征， y表带输出目标值。
        epochs: 训练轮数
        min_batch_size: 每轮训练样本数
        eta: 学习速率
        test: 测试集
        """
        test = list(test)
        train = list(train)
        if test:
            n_test = len(test)
        n_train = len(train)
        for epoch in range(epochs):
            random.shuffle(train)
            min_batches = [train[index:index+min_batch_size] for index in range(0, n_train, min_batch_size)]
            for mini_batch in min_batches:
                self.update_mini_batch(mini_batch, eta)
            if test:
                print("Epoch {0}: {1} / {2}".format(epoch, self.evaluate(test), n_test))
            else:
                print("Epoch {0} 结束。".format(epoch))
    
    def update_mini_batch(self, mini_batch, eta):
        """
        用小批量随机梯度下降和后向传播算法更新参数weights和biases
        mini_batch: 小批量数据
        eta: 学习速率
        """
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_w, delta_nabla_b = self.backprop(x, y)
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
        self.weights = [w - (eta/len(mini_batch))*nw 
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b - (eta/len(mini_batch))*nb 
                       for b, nb in zip(self.biases, nabla_b)]
    
    def backprop(self, x, y):
        """
        x: 单个样本的特征值
        y: 单个样本的目标值
        return: tuple(nabla_w, nabla_b)
        """
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        # feedforward
        activation = x
        # list to store all the activations, layer by layer
        activations = [x]
        # list to store all the z vectors, layer by layer
        zs = []
        for w, b in zip(self.weights, self.biases):
            #print("w b的维度分别是：{0}, {1}".format(w.shape, b.shape))
            z = np.dot(w, activation) + b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # error backward pass
        delta = self.cost_derivative(activations[-1], y) * sigmoid(zs[-1])
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        nabla_b[-1] = delta
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
            nabla_b[-l] = delta
        return (nabla_w, nabla_b)
    
    def evaluate(self, test):
        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test]
        return sum(int(x == y) for (x, y) in test_results)
    
    def cost_derivative(self, output_activations, y):
        """
        return: vector,  损失函数C_x对输出层神经元的输出a的偏导数
        """
        return (output_activations - y)
    
def sigmoid(z):
    """
    z: 范围[0, 1], 神经元线下加权值 z = w*x + b
    """
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """
    return: vector sigmoid函数的导数
    """
    return sigmoid(z)*(1-sigmoid(z))

In [70]:
import mnist_loader
train, validation, test = mnist_loader.load_data_wrapper()

In [71]:
netModel = neuralNetWork([784, 30, 10])

In [72]:
netModel.SGD(train, 30, 10, 3.0, test=test)

Epoch 0: 6615 / 10000
Epoch 1: 8234 / 10000
Epoch 2: 8299 / 10000
Epoch 3: 8279 / 10000
Epoch 4: 8320 / 10000
Epoch 5: 8380 / 10000
Epoch 6: 8412 / 10000
Epoch 7: 8381 / 10000
Epoch 8: 8416 / 10000
Epoch 9: 8412 / 10000
Epoch 10: 8410 / 10000
Epoch 11: 8433 / 10000
Epoch 12: 8414 / 10000
Epoch 13: 8474 / 10000
Epoch 14: 8438 / 10000
Epoch 15: 8476 / 10000
Epoch 16: 8413 / 10000
Epoch 17: 8426 / 10000
Epoch 18: 8488 / 10000
Epoch 19: 8503 / 10000
Epoch 20: 8494 / 10000
Epoch 21: 8490 / 10000
Epoch 22: 8444 / 10000
Epoch 23: 8462 / 10000
Epoch 24: 8481 / 10000
Epoch 25: 8503 / 10000
Epoch 26: 8529 / 10000
Epoch 27: 8500 / 10000
Epoch 28: 8495 / 10000
Epoch 29: 8472 / 10000


In [1]:
"""
mnist_svm
~~~~~~~~~
A classifier program for recognizing handwritten digits from the MNIST
data set, using an SVM classifier."""

#### Libraries
# My libraries
import mnist_loader 

# Third-party libraries
from sklearn import svm

def svm_baseline():
    training_data, validation_data, test_data = mnist_loader.load_data()
    # train
    clf = svm.SVC()
    clf.fit(training_data[0], training_data[1])
    # test
    predictions = [int(a) for a in clf.predict(test_data[0])]
    num_correct = sum(int(a == y) for a, y in zip(predictions, test_data[1]))
    print("Baseline classifier using an SVM.")
    print("%s of %s values correct." % (num_correct, len(test_data[1])))

ImportError: No module named 'scipy'